Qwen - features. Autoregressive on all outputs. not backward compatible due to changes in feature extraction

In [1]:
from unsloth import FastModel
import torch
from torch.nn.utils import clip_grad_value_
import torch.nn as nn
# from transformers.models.gemma3.modeling_gemma3 import Gemma3TextScaledWordEmbedding
from functions import *
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    # model_name="unsloth/gemma-3-12b-pt",
    # model_name="unsloth/gemma-3-4b-pt",
    max_seq_length = 8192, # Choose any for long context!
    load_in_4bit = True,
    resize_model_vocab=48,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
model.lm_head.load_state_dict(torch.load("../../Model/qwen48.pth"))
model.model.embed_tokens.load_state_dict(torch.load("../../Model/qwen48.pth"))
# model.lm_head.load_state_dict(torch.load("../../Model/lm_heads_weights_feature0.pth"))
model.train();
model.lm_head.weight.requires_grad_(True);
model.model.embed_tokens.weight.requires_grad_(True);

In [3]:
feature_model = FeatureEmbedding(model.model.embed_tokens, model.config).to('cuda')
# feature_model.load_state_dict(torch.load("../../Model/feature_model0.pth"))

In [4]:
import json
output_path = '/home/zhenlan/Desktop/Projects/ARC2/Data/ARC-AGI-2-main/combined_data.json'
with open(output_path, 'r') as f:
    data = json.load(f)

In [5]:
dataset_train = CustomDataset(data, True, tokenize_features, max_length=8192, IsDecode=False, max_k=5)
dataset_test = CustomDataset(data, False, tokenize_features, max_length=8192, IsDecode=False, max_k=5)
dataloader_train = DataLoader(dataset_train, batch_size=1, shuffle=True, num_workers=8)
dataloader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, num_workers=8)

In [6]:
# %debug
# inputs = next(iter(data_gen(data, False, tokenize_features, max_length=8192, IsDecode=False, max_k=5)))
# inputs = next(iter(dataloader_train))

#### Fine-tune embedding

In [7]:
epochs = 10
accumulation_steps = 32
lr = 3e-5
clip = 3e-3
MAX_LEN = 3600

In [8]:
trainable_params = list(model.lm_head.parameters()) + list(feature_model.parameters())
optimizer = torch.optim.AdamW(trainable_params,lr = lr)
loss_fn = torch.nn.CrossEntropyLoss()
print(len(trainable_params))

11


In [9]:
start_time = time.time()
train_loss = 0
val_loss = 0
for epoch in range(epochs):
    model.train()
    for i,(input_tokens, target_tokens, input_features, output_features) in enumerate(dataloader_train):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            y = target_tokens.to('cuda')
            input_features, output_features, input_tokens = input_features.to('cuda'), output_features.to('cuda'), input_tokens.to('cuda')
            inputs_embeds = feature_model(input_features, output_features, input_tokens)
            yhat = model(inputs_embeds=inputs_embeds).logits[:,:,:15]
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,(input_tokens, target_tokens, input_features, output_features) in enumerate(dataloader_test):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                y = target_tokens.to('cuda')
                input_features, output_features, input_tokens = input_features.to('cuda'), output_features.to('cuda'), input_tokens.to('cuda')
                inputs_embeds = feature_model(input_features, output_features, input_tokens)
                yhat = model(inputs_embeds=inputs_embeds).logits[:,:,:15]
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0
end_time = time.time()
print(f"Training completed in {(end_time - start_time)/60:.2f} minutes")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!
Epoch 1 - Train Loss: 2.9999 - Val Loss: 2.0828
Epoch 2 - Train Loss: 2.0537 - Val Loss: 1.8422
Epoch 3 - Train Loss: 1.8173 - Val Loss: 1.4932
Epoch 4 - Train Loss: 1.5129 - Val Loss: 1.2683
Epoch 5 - Train Loss: 1.3284 - Val Loss: 1.0732
Epoch 6 - Train Loss: 1.0486 - Val Loss: 0.7913
Epoch 7 - Train Loss: 0.8975 - Val Loss: 0.7123
Epoch 8 - Train Loss: 0.7930 - Val Loss: 0.6086
Epoch 9 - Train Loss: 0.7070 - Val Loss: 0.5528
Epoch 10 - Train Loss: 0.6651 - Val Loss: 0.5159
Training completed in 150.88 minutes


In [10]:
torch.save(model.lm_head.state_dict(), '../../Model/lm_heads_weights_feature0.pth')
torch.save(feature_model.state_dict(), '../../Model/feature_model0.pth')

#### Fine-Tune QLORA

In [11]:
epochs = 50
accumulation_steps = 32
lr = 4e-5
clip = 4e-3
MAX_LEN = 3600

In [12]:
model.max_seq_length = MAX_LEN

In [13]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 64,           # Larger = higher accuracy, but might overfit
    lora_alpha = 64,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [14]:
model.lm_head.weight.requires_grad_(True);
model.model.model.embed_tokens.weight.requires_grad_(True);
trainable_params = [param for param in model.parameters() if param.requires_grad] + [param for param in feature_model.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(trainable_params,lr = lr)
loss_fn = torch.nn.CrossEntropyLoss()
print(len(trainable_params))

516


In [15]:
start_time = time.time()
train_loss = 0
val_loss = 0
for epoch in range(epochs):
    model.train()
    for i,(input_tokens, target_tokens, input_features, output_features) in enumerate(dataloader_train):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            y = target_tokens.to('cuda')
            input_features, output_features, input_tokens = input_features.to('cuda'), output_features.to('cuda'), input_tokens.to('cuda')
            inputs_embeds = feature_model(input_features, output_features, input_tokens)
            yhat = model(inputs_embeds=inputs_embeds).logits
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,(input_tokens, target_tokens, input_features, output_features) in enumerate(dataloader_test):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                y = target_tokens.to('cuda')
                input_features, output_features, input_tokens = input_features.to('cuda'), output_features.to('cuda'), input_tokens.to('cuda')
                inputs_embeds = feature_model(input_features, output_features, input_tokens)
                yhat = model(inputs_embeds=inputs_embeds).logits
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0
end_time = time.time()
print(f"Training completed in {(end_time - start_time)/60:.2f} minutes")

Epoch 1 - Train Loss: 0.6700 - Val Loss: 0.5024
Epoch 2 - Train Loss: 0.5995 - Val Loss: 0.4603
Epoch 3 - Train Loss: 0.5646 - Val Loss: 0.4354
Epoch 4 - Train Loss: 0.5410 - Val Loss: 0.4141
Epoch 5 - Train Loss: 0.5240 - Val Loss: 0.4057


KeyboardInterrupt: 

In [19]:
start_time = time.time()
train_loss = 0
val_loss = 0
for epoch in range(epochs):
    model.train()
    for i,(input_tokens, target_tokens, input_features, output_features) in enumerate(dataloader_train):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            y = target_tokens.to('cuda')
            input_features, output_features, input_tokens = input_features.to('cuda'), output_features.to('cuda'), input_tokens.to('cuda')
            inputs_embeds = feature_model(input_features, output_features, input_tokens)
            yhat = model(inputs_embeds=inputs_embeds).logits
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,(input_tokens, target_tokens, input_features, output_features) in enumerate(dataloader_test):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                y = target_tokens.to('cuda')
                input_features, output_features, input_tokens = input_features.to('cuda'), output_features.to('cuda'), input_tokens.to('cuda')
                inputs_embeds = feature_model(input_features, output_features, input_tokens)
                yhat = model(inputs_embeds=inputs_embeds).logits
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0
end_time = time.time()
print(f"Training completed in {(end_time - start_time)/60:.2f} minutes")

Epoch 1 - Train Loss: 0.5338 - Val Loss: 0.4018
Epoch 2 - Train Loss: 0.5097 - Val Loss: 0.3952
Epoch 3 - Train Loss: 0.5027 - Val Loss: 0.3838
Epoch 4 - Train Loss: 0.4926 - Val Loss: 0.3803
Epoch 5 - Train Loss: 0.4874 - Val Loss: 0.3765
Epoch 6 - Train Loss: 0.4814 - Val Loss: 0.3791
Epoch 7 - Train Loss: 0.4774 - Val Loss: 0.3748
Epoch 8 - Train Loss: 0.4707 - Val Loss: 0.3730
Epoch 9 - Train Loss: 0.4662 - Val Loss: 0.3720
Epoch 10 - Train Loss: 0.4633 - Val Loss: 0.3655
Epoch 11 - Train Loss: 0.4584 - Val Loss: 0.3682


KeyboardInterrupt: 

In [17]:
model.save_pretrained("../../Model/merged_model_feature0")
torch.save(feature_model.state_dict(), '../../Model/feature_model0.pth')

/home/zhenlan/anaconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [6]:
epochs = 60
accumulation_steps = 32
lr = 3.5e-5
clip = 3.5e-3
MAX_LEN = 3600
model.max_seq_length = MAX_LEN

In [6]:
model = PeftModel.from_pretrained(model, "../../Model/merged_model_feature", is_trainable=True)

In [8]:
trainable_params = [param for param in model.parameters() if param.requires_grad] + [param for param in feature_model.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(trainable_params,lr = lr)
loss_fn = torch.nn.CrossEntropyLoss()
print(len(trainable_params))

513


In [9]:
start_time = time.time()
train_loss = 0
val_loss = 0
for epoch in range(epochs):
    model.train()
    for i,(input_tokens, target_tokens, input_features, output_features) in enumerate(dataloader_train):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            y = target_tokens.to('cuda')
            input_features, output_features, input_tokens = input_features.to('cuda'), output_features.to('cuda'), input_tokens.to('cuda')
            inputs_embeds = feature_model(input_features, output_features, input_tokens)
            yhat = model(inputs_embeds=inputs_embeds).logits
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,(input_tokens, target_tokens, input_features, output_features) in enumerate(dataloader_test):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                y = target_tokens.to('cuda')
                input_features, output_features, input_tokens = input_features.to('cuda'), output_features.to('cuda'), input_tokens.to('cuda')
                inputs_embeds = feature_model(input_features, output_features, input_tokens)
                yhat = model(inputs_embeds=inputs_embeds).logits
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0
end_time = time.time()
print(f"Training completed in {(end_time - start_time)/60:.2f} minutes")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!
Epoch 1 - Train Loss: 0.4133 - Val Loss: 0.2870
Epoch 2 - Train Loss: 0.3664 - Val Loss: 0.2631
Epoch 3 - Train Loss: 0.3525 - Val Loss: 0.2511
Epoch 4 - Train Loss: 0.3362 - Val Loss: 0.2467
Epoch 5 - Train Loss: 0.3244 - Val Loss: 0.2406
Epoch 6 - Train Loss: 0.3186 - Val Loss: 0.2339
Epoch 7 - Train Loss: 0.3020 - Val Loss: 0.2323
Epoch 8 - Train Loss: 0.2990 - Val Loss: 0.2318
Epoch 9 - Train Loss: 0.2966 - Val Loss: 0.2270
Epoch 10 - Train Loss: 0.2862 - Val Loss: 0.2214
Epoch 11 - Train Loss: 0.2845 - Val Loss: 0.2233
Epoch 12 - Train Loss: 0.2848 - Val Loss: 0.2181
Epoch 13 - Train Loss: 0.2692 - Val Loss: 0.2172
Epoch 14 - Train Loss: 0.2705 - Val Loss: 0.2105
Epoch 15 - Train Loss: 0.2715 - Val Loss: 0.2123
Epoch 16 - Train Loss: 0.2673 - Val Loss: 0.2140
Epoch 17 - Train Loss: 0.2628 - Val Loss: 0.2115
Epoch 18 - Train Loss: 0.2559 - Val Loss: 0.2109
Epoch 19 - Train Loss: 0.2502 - Val Loss: 0.2099
Epoch 20 - Train Loss: 0

KeyboardInterrupt: 

In [10]:
model.save_pretrained("../../Model/merged_model_feature")
torch.save(feature_model.state_dict(), '../../Model/feature_model.pth')

/home/zhenlan/anaconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


#### Decoding

In [5]:
model = PeftModel.from_pretrained(model, "../../Model/merged_model_feature", is_trainable=True)
# model = model.merge_and_unload()
FastLanguageModel.for_inference(model);

In [6]:
decoder = CausalDecoder(model, multiplier=1.2, prob_threshold=0.5, max_num_path=5)

In [7]:
start_time = time.time()
best_results, any_results = [], []
for inputs_t in data_gen(data, IsTrain=False, max_length=8192, autoregressive=True, NeedPosition=False, tokenize_func=tokenize_causal, IsDecode=True):
    decoder.decode(inputs_t,False);
    targets = inputs_t['target_tokens']
    a,b = check(decoder,targets)
    best_results.append(a)
    any_results.append(b)
    print(a,b)
end_time = time.time()
print(f"Best results: {sum(best_results)/len(best_results)}")
print(f"Any results: {sum(any_results)/len(any_results)}")
print(f"Best results exact match: {sum([res == 1 for res in best_results])/len(best_results)}")
print(f"Any results exact match: {sum([res == 1 for res in any_results])/len(any_results)}")
print(f"Completed in {(end_time - start_time)/60:.2f} minutes")

0.9476813317479191 0.9476813317479191
0.6849999999999999 0.6849999999999999
0 0
0 0
0.3888888888888889 0.6140350877192983
0 0
0.9081632653061223 0.9107142857142857
0.9288888888888889 0.9511111111111111
0.8412698412698413 0.8412698412698413
0.9751552795031057 0.9772256728778468
0.7277777777777777 0.7277777777777777
0 0.375
0.9333333333333332 0.9333333333333332
0.9577777777777778 0.9577777777777778
0.7120743034055728 0.7585139318885449
0 0
0.8600823045267489 0.8600823045267489
0.8544444444444445 0.861111111111111
0.8699999999999999 0.8800000000000001
0 0
0.8095238095238095 0.8095238095238095
0 0
0.8305785123966942 0.8305785123966942
0.7823275862068966 0.8491379310344828
0.8552631578947368 0.8552631578947368
0 0
0 0.8727810650887574
0.9765625 0.9765625
0.6833333333333333 0.6833333333333333
0.8 0.8323809523809524
0.7638888888888888 0.8333333333333334
0.97 0.97
0.75 0.75
0 0
0.8222222222222223 0.8222222222222223
0 0
0 0
0.98125 0.98125
0.9199999999999999 0.9199999999999999
0 0
0.94173602853